In [1]:
import sys
sys.path.append( '../')
import utils
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import glob

 
Music21 v.4 is the last version that will support Python 2.
Please start using Python 3 instead.

Set music21.environment.UserSettings()['warnings'] = 0
to disable this message.

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
tf.reset_default_graph()

learning_rate = 1e-3

BATCH_SIZE = 2**3
epochs=1

# length of the sequence generated
TIMESTEPS=50

# files to generate the dataset
midi_files = glob.glob('../Sounds_for_training/*.mid')[:2]

dataset_string,offset=utils.concat_dataset(midi_files)
print(1)
dataset_number,unique_dic=utils.create_dataset_number(dataset_string)
print(2)
dataset_number=np.hstack((dataset_number,offset.reshape(-1,1)))


# normalize the dataset
mean1=np.mean(dataset_number[:,0])
std1=np.std(dataset_number[:,0])
dataset_number[:,0]-=mean1
dataset_number[:,0]/=std1
mean2=np.mean(dataset_number[:,1])
std2=np.std(dataset_number[:,1])
dataset_number[:,1]-=mean2
dataset_number[:,1]/=std2

x_train=utils.get_x(dataset_number,TIMESTEPS)
print(3)
X_G = tf.placeholder("float", [None, x_train.shape[1],1])
X_real = tf.placeholder("float", [None, x_train.shape[1],x_train.shape[2]])

def noise(size):
    return np.random.randn(size,TIMESTEPS,1)

def generator(X_G): 
    with tf.variable_scope('generator',reuse=False):
        x = tf.layers.conv1d(inputs=X_G, filters=16, kernel_size=3, padding='same',activation=tf.nn.elu)
        x = tf.layers.conv1d(inputs=x, filters=32, kernel_size=3, padding='same',activation=tf.nn.elu)
        x = tf.layers.conv1d(inputs=x, filters=128, kernel_size=3, padding='same',activation=tf.nn.elu)
        x = tf.layers.conv1d(inputs=x, filters=2, kernel_size=3, padding='same')
        return x

def discriminator(x,reuse=False):
    with tf.variable_scope('discriminator',reuse=reuse):
        x = tf.layers.conv1d(inputs=x, filters=32, kernel_size=3, padding='same',activation=tf.nn.elu)
        x = tf.layers.conv1d(inputs=x, filters=64, kernel_size=3, padding='same',activation=tf.nn.elu)
        x = tf.layers.conv1d(inputs=x, filters=128, kernel_size=3, padding='same',activation=tf.nn.elu)
        x=tf.layers.dense(x,128,activation=tf.nn.elu)
        x=tf.layers.dense(x,1)
        return x


#generate a sequence when giving some random noise
output_g=generator(X_G)

# should tell it's a fake sequence because we are giving the ouput of the generator
output_d_fake=discriminator(output_g)

# should tell it's a true sequence because we are giving a true sequence
output_d_real=discriminator(X_real,reuse=True)

T_vars = tf.trainable_variables()

#list to train only the generator layers
var_list_G = [var for var in T_vars if var.name.startswith('generator')]

#list to train only the discriminator layers
var_list_D = [var for var in T_vars if var.name.startswith('discriminator')]


#the diagonale in the the confusion matrix
#compute the error of missclassified image (it says it's false instead of true) 
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_d_real, labels=tf.ones_like(output_d_real)))
#compute the error of missclassified image (it says it's true instead of false) 
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_d_fake, labels=tf.zeros_like(output_d_fake)))

#discriminator's error
loss1 = D_loss_real + D_loss_fake

#generator's error
loss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_d_fake, labels=tf.ones_like(output_d_fake)))

# we train both discrimantor and generator
optimizer1 = tf.train.AdamOptimizer(learning_rate).minimize(loss1,var_list=var_list_D)
optimizer2 = tf.train.AdamOptimizer(learning_rate).minimize(loss2,var_list=var_list_G)

z_sample=np.random.rand(1,TIMESTEPS,1)
sess=tf.Session()
sess.run(tf.global_variables_initializer())
val_loss_list = []   
for epoch in range(0,epochs):
#     we train the discriminator
    loss1_count=0
    loss2_count=0
    for batch_i in range(0, x_train.shape[0], BATCH_SIZE):
        _,loss_1 = sess.run([optimizer1,loss1], feed_dict={
            X_G: noise(x_train[batch_i:batch_i + BATCH_SIZE].shape[0]),
            X_real: x_train[batch_i:batch_i + BATCH_SIZE]})
        loss1_count+=loss_1
    # we train the generator
    for batch_i in range(0, x_train.shape[0], BATCH_SIZE):
        _,loss_2 = sess.run([optimizer2,loss2], feed_dict={X_G: noise(BATCH_SIZE)})
        loss2_count+=loss_2    
    print('Discriminator loss :',loss1_count,'Generator loss :',loss2_count)

1
2
3
('Discriminator loss :', 63.98830530216219, 'Generator loss :', 15.179629388798507)


In [6]:
#generate noise for the generator in order to generate a sequence
z_sample=np.random.rand(1,TIMESTEPS,1)
#generate the sequence with the nn above
a=sess.run(output_g,feed_dict={X_G:z_sample})
#get the result
music_generated=a[0]
#unormalize
music_generated[:,0]*=std1
music_generated[:,0]+=mean1
music_generated[:,1]*=std2
music_generated[:,1]+=mean2

#transform the result to a midi file
utils.array_to_midi(music_generated,unique_dic,'test0')

[37.227974    0.16638172]
[34.58263     0.15889525]
[44.77153     0.33934122]
[45.38925     0.32347846]
oui
[29.06872     0.21631223]
oui
[35.040817   0.1625028]
[47.607983   0.4248582]
oui
[40.61753     0.23517792]
[48.584084    0.41118085]
oui
[38.36093     0.26770455]
[51.67627     0.44438764]
[49.188896   0.3144213]
[34.347404    0.23737258]
[46.006096    0.32732904]
[36.30009     0.19922239]
[47.04889     0.26603717]
[37.600124    0.20318826]
[40.26529    0.3374461]
[40.04942     0.31681257]
[45.112526    0.22763713]
[31.171186    0.11333027]
[35.306797    0.26980925]
[43.595398    0.28715956]
[43.44748     0.20180956]
[34.42011     0.20524257]
[43.958363    0.32682514]
[44.472443   0.2722245]
[35.127613    0.22124116]
[46.897255    0.30583215]
[40.40229     0.20125861]
[53.29831     0.30765125]
[35.194283    0.26004392]
oui
[38.78119     0.24298127]
oui
[32.19472     0.19509111]
[48.79071     0.44625938]
oui
[28.522644    0.15475398]
[55.082573    0.46286583]
[53.1081      0.2763